In [1]:
import json, wandb
from importlib import reload

In [2]:
with open('experiments/todo/0GCN.json') as file:
        construct_dict = json.load(file)

In [3]:
construct_dict['data_params']['n_data']

1000000.0

In [4]:
# !wandb login b5b917a9390932e56fccfcbff6f528ccd85c44bf
# run=wandb.init(project="test", entity="chri862z", config=construct_dict)

In [5]:
construct_dict

{'experiment': 'IceCube',
 'group': 'GCN_nlayers',
 'model_name': 'GCN_nlayers',
 'angle_unitvec': 'angle',
 'wandblog': True,
 'run_params': {'epochs': 100,
  'batch_size': 512,
  'early_stop': True,
  'patience': 20,
  'lr_schedule': 'classic',
  'learning_rate': 0.001,
  'warm_up': 3,
  'lr_decay': 0.9,
  'val_epoch': 20,
  'loss_func': 'abs_vonMises_angle',
  'metrics': 'energy_angle',
  'performance_plot': 'performance_e_alpha'},
 'hyper_params': {'n_out': 4,
  'hidden_states': 64,
  'conv_layers': 0,
  'conv_activation': 'relu',
  'decode_layers': 3,
  'decode_activation': 'relu',
  'dropout': 0.1,
  'regularization': 'l2',
  'batch_norm': True,
  'forward': True,
  'edgeconv': 1},
 'data_params': {'n_data': 1000000.0,
  'graph_construction': 'classic',
  'muon': True,
  'n_neighbors': 6,
  'data_split': [0.8, 0.1, 0.1],
  'restart': True,
  'transform_path': '../db_files/muongun/transformers.pkl',
  'db_path': '../db_files/muongun/rasmus_classification_muon_3neutrino_3mio.db',
 

In [18]:
import dev.data_load as dl
reload(dl)
data=dl.graph_data

Not notebook


In [19]:
data(restart=0)

Connecting to db-file
Loading Muons
Reading files


C:\Users\chris\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\chris\anaconda3\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator RobustScaler from version 0.19.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 143.22it/s]

Splitting data to events
     dom_x    dom_y    dom_z      time  charge_log10
0 -0.26890  0.35424 -0.34950  0.344943      0.444979
1 -0.26890  0.35424 -0.48567  0.140052     -0.897229
2 -0.34788  0.45152 -0.50262 -0.070026      0.791982
3 -0.22409  0.47086 -0.33159 -0.155243      0.083393
4 -0.22409  0.47086 -0.39968 -0.045943      0.508351
   energy_log10    zenith   azimuth
0      2.422178  0.914194  0.967663
Generating adjacency matrices
Saving dataset
Loading data to memory


graph_data(n_graphs=1)

In [ ]:
from dev.utils import list_experiments, check_dataset

In [ ]:
check_dataset(construct_dict['data_params']['database'], construct_dict['data_params']['muon'], construct_dict['data_params']['n_data'], construct_dict['data_params']['graph_construction'])

In [ ]:
%run run_trainings.py